In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))  # Adjust as needed
import pandas as pd
import numpy as np
from scripts.parallel_topic_model import clean_dataframe
from bertopic import BERTopic

chosen_dataset = 'ukraine_tweets_en'
chosen_dataset = 'cop26_tweets_en'
chosen_dataset = 'covid_tweets_en'
ur_df = pd.read_parquet('./../../data/raw/'+chosen_dataset+'.parquet')
doc_info = pd.read_csv('./../../data/processed/document_info_'+chosen_dataset+'.csv')[['Document', 'Topic', 'Representative_document','Name']]
doc_info.Topic = doc_info.Topic.astype(int)
embeddings = np.load('./../../data/processed/'+chosen_dataset+'.parquet.npy')
unique_docs, unique_embeddings = clean_dataframe(ur_df, embeddings, 'text')
topic_model = BERTopic.load("./../../models/cop26_tweets_en.parquet.topic_model", embedding_model="all-MiniLM-L6-v2")

/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-31 09:41:47,559 - INFO - RANDOM_SEEDS = [1516066219, 207027981, 327446669, 272759322, 1118250615]
2025-07-31 09:41:55,488 - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2025-07-31 09:41:55,856 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 307 0
2025-07-31 09:41:55,927 - DEBUG - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json HTTP/1.1" 200 0
2025-07-31 09:41:56,169 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.

In [ ]:
from transformers import pipeline
from bertopic.representation import TextGeneration


prompt = "I have a topic described by the following keywords: [KEYWORDS]. Based on the previous keywords, what is this topic about?"

# Create your representation model
generator = pipeline('text2text-generation', model='google/flan-t5-base')
flan_model = TextGeneration(generator)


/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


In [ ]:
from bertopic.representation import MaximalMarginalRelevance

# MMR Model
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# Text generation 
representation_model = {
    "MMR":mmr_model,
    "Flan": flan_model
    }

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

topic_model.update_topics(
    unique_docs,
    topics = doc_info.Topic.to_list(),
    vectorizer_model=vectorizer_model,
    representation_model=representation_model
    )


2025-07-29 18:42:17,107 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.
100%|██████████| 251/251 [03:04<00:00,  1.36it/s]


In [11]:
for r in topic_model.get_topic_info()["Zephyr"]:
    print(r)

['Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19, Covid19,', '', '', '', '', '', '', '', '', '']
['justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin trudeau justin 

In [5]:
for r in doc_info.query("Topic == 2")["Document"]:
    print(r)

How much does Tamiflu / Oseltamivir cost in Canada? https://t.co/geGOsaA0Nn
In 2019, we began tackling rural crime in Alberta and fixing the Justice system to be more responsive toward victims of crime.

We started by announcing stiffer fines and penalties in response to illegal trespassing by anti-farming militants. https://t.co/j2EQhmFHgG
@AdrianW123 @AndrewScheer Precisely.

It really says alot that they feel a need to bullshit us, even on this (a point that really doesn't matter)... which means it appears that a daily dose of Grade A Alberta Bovine Fecal Matter will be on the CPC menu for some time yet.

They just can't stop lying!
#Alberta Premier Jason Kenney and Deputy Prime Minister @CaFreeland will hold a one-on-one meeting today 

#Abpoli
https://t.co/h7rtLlmME8
2nd day in a row that the NEW @CTV_PowerPlay  (with @EvanLSolomon ) is NOT featuring Political Strategists on Panels to debate issues. Instead they are having a DOUBLE DOSE (TWO segments) of the press gallery featurin

KeyboardInterrupt: 

## Generation with Zephyr-7B-α
Failed because of uncaught error in transformers.pipeline:
Maybe because of ctransformer model.
The huggingface model is too big.
```python 
AttributeError: 'TextGenerationPipeline' object has no attribute 'assistant_model'
```


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

generator = pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

print(generator("Once upon a time"))


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [1]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from bertopic.representation import TextGeneration
from bertopic.representation import MaximalMarginalRelevance

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q3_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")


/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 5833.52it/s]
CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits fr

In [1]:
!pip install ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

from ctransformers import AutoModelForCausalLM
#from transformers import AutoTokenizer, pipeline
import transformers

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

# Pipeline
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-iok0rq9r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-iok0rq9r
  Resolved https://github.com/huggingface/transformers to commit c4e20698985887215f7e91a02621265f047af2d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 6026.30it/s]
CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits fr

AttributeError: 'TextGenerationPipeline' object has no attribute 'assistant_model'

In [ ]:

# MMR Model
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Zephyr
zephyr = TextGeneration(generator, prompt=prompt)
representation_model = {
    "MMR":mmr_model,
    "Zephyr": zephyr
    }


In [ ]:

topic_model.update_topics(unique_docs, topics = doc_info.Topic.to_list(),
                          representation_model=representation_model )


